#### Business Analytics FHDW 2025
# Ein erstes kleines Beispiel zur logistischen Regression
## zur Illustration praktischer Aspekte

Wir betrachten den bekannten Datensatz *UniversalBank.csv* und führen damit eine einfache logistische Regression durch. Sie soll auf Basis des Einkommens voraussagen, ob ein Kunde/eine Kundin ein Angebot über ein Darlehen annehmen würde.

Zunächst bereiten wir den Datensatz auf die bereits bekannte Weise vor.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pylab as plt

from dmba import classificationSummary

bank_df = pd.read_csv('./Daten/UniversalBank.csv')
print(bank_df.head)

y = bank_df['PersonalLoan']
X = bank_df[['Income']]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

Die Durchführung der logistischen Regression über die Anpassung an die Trainingsdaten mit `fit` ist dann auch gewohnt einfach:

In [ ]:
lr = LogisticRegression()
lr.fit(train_X, train_y)

Wir konstruieren uns daraus einmal selbst die Funktion zur Bestimmung der bedingten Wahrscheinlichkeiten. Wir nutzen nur einen Prädiktor, also haben wir $p(loan = yes | income = x) = \frac{1}{1+e^{-(\beta_0 + \beta_1 x)}}$.

In [ ]:
intercept = lr.intercept_[0]
coeff = lr.coef_[0][0]

x = np.arange(0, 250, 1) # Skaliert auf Basis der Einkommensspanne
P = 1/(1+np.exp(-1*(intercept+coeff*x)))

print(intercept, coeff)

Für die gesuchten Betas greifen wir auf die Attribute der `LogisticRegression` zu. $\beta_0$ finden wir in `intercept_`, $\beta_1$ in `coef_`. Unsere angepasste Funktion ist dann $p(loan = yes | income = x) = \frac{1}{1+e^{6.063 - 0.036 x}}$. Werfen wir einen Blick darauf, zusammen mit den Darlehensentscheidungen aus dem Datensatz:

In [ ]:
plt.plot(x,P)
plt.scatter(bank_df['Income'], bank_df['PersonalLoan'], s=10)
plt.title('Wahrscheinlichkeiten und tatsächliche Entscheidungen für Darlehen')
plt.xlabel('Einkommen')
plt.ylabel('P für Darlehen')
plt.show()

Als Chancen bzw. Odds können wir das auch darstellen:

In [ ]:
Odds = np.exp(intercept+coeff*x)
       
plt.plot(x,Odds)
plt.title('Chancen für Annahme Darlehen')
plt.xlabel('Einkommen')
plt.ylabel('Odds')
plt.show()

Bis jetzt haben wir uns mit den bedingten Wahrscheinlichkeiten beschäftigt. Ursprünglich wollen wir mit der logistischen Regression aber eine binäre Prognose erhalten, in diesem Fall *Darlehen ja oder nein*. Dazu können wir wieder die Funktion `predict` nutzen. Für die Beurteilung der Performance nutzen wir hier eine Konfusionsmatrix, jeweils für die Trainings- und die Validierungsdaten.

In [ ]:
classificationSummary(train_y, lr.predict(train_X))
print('')
classificationSummary(valid_y, lr.predict(valid_X))

Um aus den Wahrscheinlichkeiten zur binären Entscheidung zu kommen, wendet die Funktion eine Schwelle bzw. den *cutoff* $c$ an. Liegt die prognostizierte Wahrscheinlichkeit eines Datensatzes auf oder über $c$, wird angenommen, dass der Kunde/die Kundin das Darlehen akzeptiert, also Ereignis $1$ eintritt, sonst $0$. Im Falle der hier genutzten Funktion `predict` von *scikit learn* besitzt $c$ einen *default*-Wert von $0.5$. Überprüfen wir das kurz selbst:

In [ ]:
train_predictions = np.where(P[train_X] >= 0.5, 1, 0)
valid_predictions = np.where(P[valid_X] >= 0.5, 1, 0)
classificationSummary(train_y, train_predictions)
print('')
classificationSummary(valid_y, valid_predictions)

Und noch eine Darstellung der ermittelten Wahrscheinlichkeitsfunktion und der Prädiktionen der Trainingsdatensätze:

In [ ]:
plt.plot(x, P, color='r')
plt.scatter(train_X['Income'], lr.predict_proba(train_X)[:,1], s=10)
plt.show()

## Aufgabe
Analysieren Sie die Auswirkungen verschiedener *cutoff*-Werte im gegebenen Modell auf geeignete Weise: Generieren Sie einen `scatter`-Graphen mit *cutoff*-Werten auf der x-Achse und der resultierenden Genauigkeit auf der y-Achse. `accuracy_score` von *scikit learn* kann dabei hilfreich sein.